In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import tensorflow.keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import random
from datetime import datetime
import time

pi = 3.14159265359

maxval=1e9
minval=1e-9

2025-06-17 19:08:15.654104: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-17 19:08:16.597014: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from dataloaders.OptimizedDataGenerator_v2 import OptimizedDataGenerator
from loss import *
from mlp_encoder_model import *

In [3]:
seed = 10
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
dataset_base_dir = "/data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/"
tfrecords_base_dir = os.path.join(dataset_base_dir, "TFR_files", "2t")

dataset_train_dir = os.path.join(dataset_base_dir, "train")
dataset_validation_dir = os.path.join(dataset_base_dir, "test")
tfrecords_dir_train = os.path.join(tfrecords_base_dir, "TFR_train_slim")
tfrecords_dir_val   = os.path.join(tfrecords_base_dir, "TFR_val_slim")

In [5]:
print(f'Number of training files: {len(os.listdir(dataset_train_dir))}')
print(f'Number of validation files: {len(os.listdir(dataset_validation_dir))}')

Number of training files: 80
Number of validation files: 20


In [6]:
batch_size = 5000
val_batch_size = 5000
train_file_size = len(os.listdir(dataset_train_dir))
val_file_size = len(os.listdir(dataset_validation_dir))

In [7]:
start_time = time.time()
validation_generator = OptimizedDataGenerator(
    dataset_base_dir = dataset_validation_dir,
    file_type = "parquet",
    data_format = "3D",
    batch_size = val_batch_size,
    file_count = val_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotBeta'],
    input_shape = (2,16,16), # (20,16,16),
    transpose = (0,2,3,1),
    shuffle = False, 
    files_from_end=True,

    tfrecords_dir = tfrecords_dir_val,
    use_time_stamps = [0,19],
    max_workers = 2,
    load_from_tfrecords_dir = tfrecords_dir_val
)

print("--- Validation generator %s seconds ---" % (time.time() - start_time))


Loading metadata from /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_val_slim/metadata.json


--- Validation generator 0.24593091011047363 seconds ---


In [8]:
# training generator
start_time = time.time()
training_generator = OptimizedDataGenerator(
    dataset_base_dir = dataset_train_dir,
    file_type = "parquet",
    data_format = "3D",
    batch_size = batch_size,
    file_count = train_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotBeta'],
    input_shape = (2,16,16), # (20,16,16),
    transpose = (0,2,3,1),
    shuffle = False, # True 

    tfrecords_dir = tfrecords_dir_train,
    use_time_stamps = [0,19],
    max_workers = 2,
    load_from_tfrecords_dir = tfrecords_dir_train
)
print("--- Training generator %s seconds ---" % (time.time() - start_time))

Loading metadata from /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_train_slim/metadata.json
--- Training generator 0.16203856468200684 seconds ---


In [9]:
training_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir = tfrecords_dir_train,
    shuffle = True,
    seed = seed,
    quantize = True
)

validation_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir = tfrecords_dir_val,
    shuffle = True,
    seed = seed,
    quantize = True
)


Loading metadata from /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_train_slim/metadata.json
Loading metadata from /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_val_slim/metadata.json


In [10]:
model=CreateModel((16,16,2))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=custom_sse_loss
)

model.summary()

2025-06-17 19:08:59.232723: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3234 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 1g.5gb, pci bus id: 0000:c1:00.0, compute capability: 8.0


Model: "smrtpxl_regression"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_pxls/ (InputLayer)    [(None, 16, 16, 2)]          0         []                            
                                                                                                  
 average_pooling2d (Average  (None, 16, 1, 2)             0         ['input_pxls/[0][0]']         
 Pooling2D)                                                                                       
                                                                                                  
 average_pooling2d_1 (Avera  (None, 1, 16, 2)             0         ['input_pxls/[0][0]']         
 gePooling2D)                                                                                     
                                                                                 

In [11]:
# training
pitch = '50x12P5'
fingerprint = '%08x' % random.randrange(16**8)
base_dir = '/home/dajiang/smart-pixels-ml/weights/dataset_3src_16x16_weights/'
weights_dir = base_dir + 'weights-{}-bs{}-{}-2t-mlp-sse-slim-checkpoints'.format(pitch, batch_size, fingerprint)

# create output directories
if os.path.isdir(base_dir):
    os.mkdir(weights_dir)
else:
    os.mkdir(base_dir)
    os.mkdir(weights_dir)
    
checkpoint_filepath = weights_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'
mcp = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=False,
)

print('Model fingerprint: {}'.format(fingerprint))

Model fingerprint: 34c2da80


In [ ]:
history = model.fit(x=training_generator,
                    validation_data=validation_generator,
                    callbacks=[mcp],
                    epochs=1000,
                    shuffle=False, # shuffling now occurs within the data-loader
                    verbose=1)

Epoch 1/1000


2025-06-17 19:09:11.476960: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8906
2025-06-17 19:09:11.486086: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-06-17 19:09:11.518851: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd418aa2f60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-06-17 19:09:11.518908: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB MIG 1g.5gb, Compute Capability 8.0
2025-06-17 19:09:11.523199: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-06-17 19:09:11.619354: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-06-17 19:09

84/84 [==============================] - 10s 84ms/step - loss: 773.3561 - val_loss: 301.3843
Epoch 2/1000
84/84 [==============================] - 7s 77ms/step - loss: 213.7697 - val_loss: 166.0788
Epoch 3/1000
84/84 [==============================] - 6s 77ms/step - loss: 143.3676 - val_loss: 125.6869
Epoch 4/1000
84/84 [==============================] - 7s 79ms/step - loss: 113.2783 - val_loss: 104.2026
Epoch 5/1000
84/84 [==============================] - 6s 76ms/step - loss: 96.2043 - val_loss: 92.0492
Epoch 6/1000
84/84 [==============================] - 6s 77ms/step - loss: 85.9248 - val_loss: 82.7736
Epoch 7/1000
84/84 [==============================] - 6s 76ms/step - loss: 78.4722 - val_loss: 76.1569
Epoch 8/1000
84/84 [==============================] - 6s 76ms/step - loss: 73.7495 - val_loss: 73.3386
Epoch 9/1000
84/84 [==============================] - 6s 75ms/step - loss: 69.1906 - val_loss: 68.5960
Epoch 10/1000
84/84 [==============================] - 6s 74ms/step - loss: 6